# Preliminaries

In [1]:
# import modules
import numpy as np
... # your code here

Ellipsis

In [2]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

In [3]:
a = np.arange(10)
print(np.random.permutation(a[:3]))

[1 2 0]


# Density Tree

In [4]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
#         print("shape of m and M", m.shape, M.shape)
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1
        
#         print("shape of valid features", valid_features.shape)

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                left, right = make_density_split_node(node, N, np.random.permutation(valid_features)[:D_try])
                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)

    def predict(self, x):        
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        leaf = self.find_leaf(x)
        m, M = self.root.box
        if np.sum(x<m)>0 or np.sum(x>M)>0:  # out of the bounding box
            return 0
        else:
            return self.prior * leaf.response

In [5]:
a = np.array([1,2,5])
b = np.array([2,3,4])
print(np.where(a<b))

(array([0, 1]),)


In [6]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        """if we don't use np.unique(), we will end up calculating the error of some identical thresholds for multiple times"""
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = np.array([(data_unique[i]+data_unique[i+1])/2 for i in range(len(data_unique)-1)])
#         print(tj)
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # Compute the error
            Nl = np.sum(node.data[:, j]<=t)
            Nr = np.sum(node.data[:, j]>t)
#             Vl = np.prod(np.max(node.data[node.data[:, j]<=t], axis=0) - np.min(node.data[node.data[:, j]<=t], axis=0))
#             Vr = np.prod(np.max(node.data[node.data[:, j]>t], axis=0) - np.min(node.data[node.data[:, j]>t], axis=0))
            V = np.prod(M-m)
            Vl = V*(t - m[j])/(M[j] - m[j])
            Vr = V*(M[j] - t)/(M[j] - m[j])
            loo_error = cal_loo_error(N, Nl, Vl) + cal_loo_error(N, Nr, Vr)
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t
    
    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    left.data = node.data[node.data[:, j_min]<=t_min] # store data in left node -- for subsequent splits
    left.box = m.copy(), M.copy()
    left.box[1][j_min] = t_min   # store bounding box in left node
    right.data = node.data[node.data[:, j_min]>t_min]
    right.box = m.copy(), M.copy()
    right.box[0][j_min] = t_min

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right


def cal_loo_error(N, Nm, Vm):
    return (Nm/(N*Vm))*(Nm/N - 2*(Nm-1)/(N-1))

In [7]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    v = np.prod(node.box[1]-node.box[0])
    node.response = n/(N*v)

# Decision Tree

In [8]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                left, right = make_decision_split_node(node, np.random.permutation(node.data.shape[1])[:D_try])
                stack.append(left)
                stack.append(right)
                ... # your code here
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node)
                ... # your code here
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response # your code here

In [9]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = np.array([(data_unique[i]+data_unique[i+1])/2 for i in range(len(data_unique)-1)])
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # Compute the error
            Nl = np.sum(node.data[:, j]<=t)
            Nr = np.sum(node.data[:, j]>t)
            gini = cal_gini(Nl, node.labels, node.data[:, j]<=t) + cal_gini(Nr, node.labels, node.data[:, j]>t)
            
            # choose the best threshold that
            if gini < e_min:
                e_min = gini
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = node.data[node.data[:, j_min]<=t_min] # data in left node
    left.labels = node.labels[node.data[:, j_min]<=t_min] # corresponding labels
    right.data = node.data[node.data[:, j_min]>t_min]
    right.labels = node.labels[node.data[:, j_min]>t_min]


    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right    

In [10]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    
    # compute and store leaf response
    node.N = len(node.labels)
    response = dict()   # Key: label  Value: K/N
    for label in node.labels:
        if label not in response:
            response[label] = 1
        else:
            response[label] += 1
    for k in response:
        response[k] = response[k] / node.N
    node.response = response # your code here

In [11]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return len(np.unique(node.labels))==1 # your code here

In [12]:
def cal_gini(Nl, labels, indices):
    """
    Calculation of Gini impurity
    Args:
        Nl: number of instances in node l
        labels: labels of node l's parent
        indices: use labels[indices] to get the labels of node l
    """
    label_count = dict()  # key: label  value: count of the label
    for label in labels[indices]:
        if label not in label_count:
            label_count[label] = 1
        else:
            label_count[label] += 1
    gini = 0
    for _, Nlk in label_count.items():
        gini += np.square(Nlk/Nl)
    gini = Nl*(1-gini)
    return gini

# Evaluation of Density and Decision Tree

In [13]:
# read and prepare the digits data
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.keys())
print(type(digits.data))
print(type(digits.images))
print(digits.images.shape)
print(digits.target.shape)
... # your code here

dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1797, 8, 8)
(1797,)


Ellipsis

In [14]:
# train trees, plot training error confusion matrices, and comment on your results
density_trees = [DensityTree() for _ in range(10)]
for i in range(10):
    density_trees[i].train(data=digits.data[digits.target==i], prior=np.sum(digits.target==i)/len(digits.target), n_min=20)
confusion_matrice = np.zeros((10, 10))
for i in range(len(digits.target)):
    x = digits.data[i]
    real_label = digits.target[i]
    pred_label = np.argsort([density_trees[i].predict(x) for i in range(10)])[-1]
#     print("real:", real_label)
#     print("pred:", [density_trees[i].predict(x) for i in range(10)])
    confusion_matrice[real_label][pred_label] += 1
print(confusion_matrice)


[[178.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 139.  15.   0.   3.   1.   0.   0.  18.   6.]
 [  0.   5. 109.   3.   0.   0.   0.   0.  60.   0.]
 [  0.   6.  12. 107.   0.  12.   0.   3.  37.   6.]
 [  0.   1.   0.   0. 152.   9.   0.  19.   0.   0.]
 [  0.   1.   0.  23.   1. 133.   0.   3.   9.  12.]
 [  0.   0.   0.   0.   0.   0. 180.   0.   1.   0.]
 [  0.   0.   0.   0.   6.   3.   0. 169.   1.   0.]
 [  0.  34.   1.   0.   0.   6.   0.   2. 131.   0.]
 [  0.   6.   3.  37.   7.   4.   0.  10.  11. 102.]]


In [15]:
decision_tree = DecisionTree()
decision_tree.train(data=digits.data, labels=digits.target, n_min=5)
confusion_matrice = np.zeros((10, 10))
for i in range(len(digits.target)):
    x = digits.data[i]
    real_label = digits.target[i]
    pred_label = 0
    max_prob = 0
    for label, p in decision_tree.predict(x).items():
        if p > max_prob:
            pred_label = label
            max_prob = p
#     print("real:", real_label)
#     print("pred:", [density_trees[i].predict(x) for i in range(10)])
    confusion_matrice[real_label][pred_label] += 1
    
print(confusion_matrice)

[[177.   0.   0.   0.   0.   1.   0.   0.   0.   0.]
 [  0. 172.   3.   0.   0.   2.   1.   0.   2.   2.]
 [  0.   0. 174.   1.   0.   0.   0.   0.   1.   1.]
 [  0.   1.   2. 177.   0.   2.   0.   0.   1.   0.]
 [  0.   2.   0.   0. 173.   2.   1.   2.   1.   0.]
 [  1.   1.   0.   1.   1. 175.   1.   0.   2.   0.]
 [  0.   0.   0.   0.   1.   1. 179.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   1.   0. 177.   0.   1.]
 [  1.   0.   3.   2.   1.   2.   2.   2. 161.   0.]
 [  0.   1.   1.   2.   0.   0.   0.   1.   0. 175.]]


In [16]:
# data = digits.data

# N, D = data.shape
# D_try = int(np.sqrt(D)) # number of features to consider for each split decision

# m, M = np.min(data, axis=0), np.max(data, axis=0)
# valid_features   = np.where(m != M)[0]
# invalid_features = np.where(m == M)[0]
# M[invalid_features] = m[invalid_features] + 1

# # initialize the root node
# root = Node()
# root.data = data
# root.box = m.copy(), M.copy()
# left, right = make_density_split_node(root, N, valid_features)
# print(left.data)
# print('\n')
# print(right.data)

# Density and Decision Forest

In [17]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [18]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [19]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here